This notebook is used to produce Figures 7-9 in the notes: steady-state solutions for some different parameter choices.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# add path to code
import sys
sys.path.insert(0, '../code')
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.fft import ifft2,fft2
from operators import compute_u,compute_w,compute_h,compute_s,h_steady,s_steady
from params import *

Define the melt-rate perturbation:

In [ ]:
sigma = 10.0/3.0                                                    # standard deviation for gaussian melt anomaly

# For Figure 8 (melting) set:
m = 5*np.exp(-0.5*(sigma**(-2))*(x**2+y**2 ))*(t_r/3.154e7)/H     # melt anomaly with amplitude of 5 m/yr

# For Figure 9 and 10 set the negative version (freeze-on):
# m = 5*np.exp(-0.5*(sigma**(-2))*(x**2+y**2 ))*(t_r/3.154e7)/H    # melt anomaly with amplitude of 5 m/yr


Define a vertical exaggeration for plotting the elevations (since these ones are a small fraction of the ice thickness):

In [ ]:
ex = 4e1   # vertical exaggeration factor

First we compute the solutions with a non-advective background state ($\alpha=0$). The only annoying thing here is interpolating the velocities onto the deformed grid defined by $z\mapsto (1-z)\gamma s + z\gamma h $ (for plotting), where $\gamma$ is the vertical exaggeration factor.

In [ ]:
# First compute the elevation solutions

alpha = 0
h = compute_h(t,k,kx,alpha,m)
s = compute_s(t,k,kx,alpha,m)
# h = h_steady(k,alpha,m)
# s = s_steady(k,alpha,m)

# # extract the centerline profiles 
yc = int(Ny/2.)     # y index at centerline         
hc = h[-1,:,yc] 
sc = s[-1,:,yc]
mc = m[-1,:,yc]



Repeat the above steps for a positive value of the background advection parameter:

In [ ]:
alpha_2 = 2e-1
m_2 = -5*np.exp(-0.5*(sigma**(-2))*(x**2+y**2 ))*(t_r/3.154e7)/H
h_2 = compute_h(t,k,kx,alpha_2,m_2)
s_2 = compute_s(t,k,kx,alpha_2,m_2)
# h_2 = h_steady(k,alpha_2,m_2)
# s_2 = s_steady(k,alpha_2,m_2)
hc_2 = h_2[-1,:,yc] 
sc_2 = s_2[-1,:,yc]
mc_2 = m_2[-1,:,yc]



Plot both solutions:

In [ ]:
plt.close()
fig = plt.figure(figsize=(14,4))

plt.subplot(121)
plt.title(r'(a) $\alpha=0$',fontsize=20,loc='left',zorder=100)
plt.plot(x0,ex*sc,linewidth=2.5,color='darkblue',zorder=16,label=r'$\gamma s$')
plt.plot(x0,ex*hc+1,linewidth=2.5,color='forestgreen',zorder=13,label=r'$1+\gamma h$')

plt.plot(x0,ex*(-delta*sc)+1,linewidth=2.5,color='k',linestyle='--',zorder=80)

plt.fill_between(x0,y1=ex*sc, y2=ex*hc+1,facecolor='aliceblue',alpha=1.0)
plt.fill_between(x0,y1=-2*np.ones(np.size(x0)), y2=ex*sc,facecolor='lightsteelblue',alpha=1,zorder=15)

plt.annotate(xy=(-L+0.1,1.03),text='air',fontsize=20,zorder=30)
plt.annotate(xy=(-L+0.1,0.05),text='ice',fontsize=20,zorder=31)
plt.annotate(xy=(-L+0.1,-0.12),text='water',fontsize=20,zorder=32)
    
plt.ylabel(r'$z$',fontsize=20)
plt.xlabel(r'$x$',fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(-0.5,1.25)
plt.xlim(-L,L)

axins = plt.gca().inset_axes([0.0, -0.5,1, 0.25],sharex=plt.gca())
axins.plot(x0,mc,linewidth=2,color='crimson',linestyle='-')
axins.set_ylabel(r'$m$',fontsize=16,color='crimson')
axins.set_ylim(-np.abs(m).max()-2e-4,np.abs(m).max()+2e-4)
axins.tick_params(axis='y', which='major', labelsize=16,color='crimson',labelcolor='crimson')
axins.tick_params(axis='x',direction="in",which='major', labelsize=16,top=True, labeltop=True, bottom=False, labelbottom=False)
axins.ticklabel_format(axis='y', style='sci', scilimits=(-4,-4),useMathText=True)
plt.setp(axins.get_xticklabels(), visible=False)

plt.subplot(122)
plt.title(r'(b) $\alpha={1}/{5}$',fontsize=20,loc='left',zorder=100)

plt.plot(x0,ex*sc_2,linewidth=2.5,color='darkblue',zorder=16,label=r'$\gamma s$')
plt.plot(x0,ex*hc_2+1,linewidth=2.5,color='forestgreen',zorder=13,label=r'$1+\gamma h$')

# # vv hydrostatic thickness profile vv
plt.plot(x0,ex*(-delta*sc_2)+1,linewidth=2.5,color='k',linestyle='--',zorder=80,label=r'$1-\gamma\delta s$')

plt.fill_between(x0,y1=ex*sc_2, y2=ex*hc_2+1,facecolor='aliceblue',alpha=1.0)
plt.fill_between(x0,y1=-2*np.ones(np.size(x0)), y2=ex*sc_2,facecolor='lightsteelblue',alpha=1,zorder=15)
plt.legend(fontsize=20,ncol=3,bbox_to_anchor=(0.55,-0.5))
plt.xlabel(r'$x$',fontsize=20)
plt.ylabel(r'$z$',fontsize=20)
plt.xticks(fontsize=16)
plt.gca().yaxis.tick_right()
plt.gca().yaxis.set_label_position("right")
plt.yticks(fontsize=16)
plt.ylim(-0.5,1.25)
plt.xlim(-L,L)

axins = plt.gca().inset_axes([0.0, -0.5,1, 0.25],sharex=plt.gca())
axins.plot(x0,mc_2,linewidth=2,color='crimson',linestyle='-')
axins.set_ylabel(r'$m$',fontsize=16,color='crimson')
axins.yaxis.set_label_position("right")
axins.set_ylim(-np.abs(m_2).max()-2e-4,np.abs(m_2).max()+2e-4)
axins.ticklabel_format(axis='y', style='sci', scilimits=(-4,-4),useMathText=True)
axins.yaxis.set_offset_position('right')
axins.tick_params(axis='y', which='major', labelsize=16,color='crimson',labelcolor='crimson',left=False, labelleft=False, right=True, labelright=True)
axins.tick_params(axis='x',direction="in",which='major', labelsize=16,top=True, labeltop=True, bottom=False, labelbottom=False)
plt.setp(axins.get_xticklabels(), visible=False)

plt.savefig('profiles',bbox_inches='tight')
plt.show()

Yay! Notes: the dashed line at the upper surface is the perfect flotation thickness and the flow speed has been normalized by its maximum absolute value.